In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-2q7ce_ac
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-2q7ce_ac
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=ddf9d9ede5c5ddb9820174ac6111a11a813e8a15ef7aa74a4b03152c8b48c4be
  Stored in directory: /tmp/pip-ephem-wheel-cache-pwwrypyz/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%writefile sum.cu
#include <iostream>
#include <vector>
#include <climits>
__global__ void min_reduction_kernel(int* arr, int size, int* result) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < size) {
atomicMin(result, arr[tid]);
}
}
__global__ void max_reduction_kernel(int* arr, int size, int* result) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < size) {
atomicMax(result, arr[tid]);
}
}
__global__ void sum_reduction_kernel(int* arr, int size, int* result) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < size) {
atomicAdd(result, arr[tid]);
}
}
__global__ void average_reduction_kernel(int* arr, int size, int* sum) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < size) {
atomicAdd(sum, arr[tid]);
}
}
int main() {
int size;
std::cout << "Enter the size of the array: ";
std::cin >> size;
std::vector<int> arr(size);
for (int i = 0; i < size; ++i) {
std::cout << "Enter element " << i << ": ";
std::cin >> arr[i];
}
int* d_arr;
int* d_result;
int result_min = INT_MAX;
int result_max = INT_MIN;
int result_sum = 0;
// Allocate memory on the device
cudaMalloc(&d_arr, size * sizeof(int));
cudaMalloc(&d_result, sizeof(int));
// Copy data from host to device
cudaMemcpy(d_arr, arr.data(), size * sizeof(int),
cudaMemcpyHostToDevice);
cudaMemcpy(d_result, &result_min, sizeof(int), cudaMemcpyHostToDevice);
// Perform min reduction
min_reduction_kernel<<<(size + 255) / 256, 256>>>(d_arr, size,
d_result);
cudaMemcpy(&result_min, d_result, sizeof(int), cudaMemcpyDeviceToHost);
std::cout << "Minimum value: " << result_min << std::endl;
// Perform max reduction
cudaMemcpy(d_result, &result_max, sizeof(int), cudaMemcpyHostToDevice);
max_reduction_kernel<<<(size + 255) / 256, 256>>>(d_arr, size,
d_result);
cudaMemcpy(&result_max, d_result, sizeof(int), cudaMemcpyDeviceToHost);
std::cout << "Maximum value: " << result_max << std::endl;
// Perform sum reduction
cudaMemcpy(d_result, &result_sum, sizeof(int), cudaMemcpyHostToDevice);
sum_reduction_kernel<<<(size + 255) / 256, 256>>>(d_arr, size,
d_result);
cudaMemcpy(&result_sum, d_result, sizeof(int), cudaMemcpyDeviceToHost);
std::cout << "Sum: " << result_sum << std::endl;
// Perform average reduction on CPU side
double average = static_cast<double>(result_sum) / size;
std::cout << "Average: " << average << std::endl;
// Free device memory
cudaFree(d_arr);
cudaFree(d_result);
return 0;
}


Writing sum.cu


In [ ]:
!nvcc sum.cu -o sum
!./sum

Enter the size of the array: 5
Enter element 0: 46
Enter element 1: 32
Enter element 2: 47
Enter element 3: 12
Enter element 4: 19
Minimum value: 12
Maximum value: 47
Sum: 156
Average: 31.2
